In [1]:
import pandas as pd
import os
import string

import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')



from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.preprocessing import OneHotEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nicolailewis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nicolailewis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nicolailewis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nicolailewis/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
! pwd

/Users/nicolailewis/code/wRajter/language_chatbot/notebooks


In [3]:
import os
import pandas as pd
import yaml
import os


def getting_yaml_data(path_to_input_file='../raw_data/small conversational bot'):
    '''
    takes a yaml format input and returns pandas dataframe
    - please, specify the path to the input file,directory
    '''

    # as this dataset consits from multiple files, we will store all the file names and open them later one by one
    files = os.listdir(path_to_input_file) # path to the directory with all the YAML files


    # creating a dictionary where we will store data form all the files
    data_dic = {
        'tag': [],
        'patterns': [],
        'responses': []
    }

    # opening the YAML files one by one and storing the content into the dictionary
    for file in files:
        if 'Zone.Identifier' not in file:
            with open(f'{path_to_input_file}/{file}', "r") as stream:
                data = yaml.safe_load(stream)

                for record in data['conversations']:
                    data_dic['tag'].append(data['categories'][0])
                    data_dic['patterns'].append(record[0])
                    data_dic['responses'].append(record[1])

    # creating a pandas dataframe form the dictionary
    return pd.DataFrame.from_dict(data_dic)

In [21]:
data = getting_yaml_data()
data

,tag,patterns,responses
0,politics,have you read the communist,"yes, marx had made some interesting observations."
1,politics,what is a government,ideally it is a representative of the people.
2,politics,what is greenpeace,global organization promoting enviornmental ac...
3,politics,what is capitalism,the economic system in which all or most of th...
4,politics,what is socialism,communism from people who want to keep their v...
...,...,...,...
561,science,chemistry,my favorite subject is chemistry
562,science,the same wavelength,it means we agree.
563,science,tell me about venus,what is venus
564,science,tell me about venus,venus is the second planet from the sun.


In [5]:
data['tag'].value_counts()

AI            105
psychology     93
humor          49
emotion        48
literature     43
money          31
science        29
greetings      25
movies         23
sports         20
profile        19
gossip         19
food           17
politics       16
trivia         10
computers      10
history         8
health          1
Name: tag, dtype: int64

In [6]:
data['tag'].unique()

array(['politics', 'AI', 'emotion', 'computers', 'profile', 'history',
       'psychology', 'food', 'literature', 'money', 'trivia', 'gossip',
       'humor', 'greetings', 'sports', 'movies', 'science', 'health'],
      dtype=object)

## Preprocessing patterns column

In [7]:
X = data['patterns']

In [8]:
#lower
def lower(X):
    X = X.str.lower()
    return X

X = lower(X)
X

0      have you read the communist
1             what is a government
2               what is greenpeace
3               what is capitalism
4                what is socialism
                  ...             
561                      chemistry
562            the same wavelength
563            tell me about venus
564            tell me about venus
565            how is your health?
Name: patterns, Length: 566, dtype: object

In [9]:
#removing punctuation and numbers
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

X = X.apply(remove_punctuations)
X

0      have you read the communist
1             what is a government
2               what is greenpeace
3               what is capitalism
4                what is socialism
                  ...             
561                      chemistry
562            the same wavelength
563            tell me about venus
564            tell me about venus
565             how is your health
Name: patterns, Length: 566, dtype: object

In [10]:
def remove_numbers(X):
    X = X.apply(lambda x: ''.join(word for word in x if not word.isdigit()))
    return X
X = remove_numbers(X)
X

0      have you read the communist
1             what is a government
2               what is greenpeace
3               what is capitalism
4                what is socialism
                  ...             
561                      chemistry
562            the same wavelength
563            tell me about venus
564            tell me about venus
565             how is your health
Name: patterns, Length: 566, dtype: object

In [11]:
stop_words = set(stopwords.words('english')) 

def remove_stop_words(X):
    word_tokens = word_tokenize(X) 
    X = [w for w in word_tokens if not w in stop_words] 
    return X
  
for index, value in X.iteritems(): 
    X[index] = remove_stop_words(value)
    
X

0      [read, communist]
1           [government]
2           [greenpeace]
3           [capitalism]
4            [socialism]
             ...        
561          [chemistry]
562         [wavelength]
563        [tell, venus]
564        [tell, venus]
565             [health]
Name: patterns, Length: 566, dtype: object

In [12]:
lemmatizer = WordNetLemmatizer()

def lemmatizing(text):
    lem = [lemmatizer.lemmatize(word) for word in text]
    new_text = ' '.join(lem)
    return new_text

for index, value in X.iteritems(): 
    X[index] = lemmatizing(value)
    
X

0      read communist
1          government
2          greenpeace
3          capitalism
4           socialism
            ...      
561         chemistry
562        wavelength
563        tell venus
564        tell venus
565            health
Name: patterns, Length: 566, dtype: object

## Preprocessing y

In [13]:
y = data[['tag']]
y.nunique()

tag    18
dtype: int64

In [20]:
enc = OneHotEncoder(sparse=False)

y_proc= enc.fit_transform(y).T

y_proc = pd.DataFrame(y_proc)

y_proc = y_proc.T

y_proc


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [24]:
y_proc.columns = data['tag'].unique()
y_proc

,politics,AI,emotion,computers,profile,history,psychology,food,literature,money,trivia,gossip,humor,greetings,sports,movies,science,health
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [27]:
y_proc.sum()

politics      105.0
AI             10.0
emotion        48.0
computers      17.0
profile        19.0
history        25.0
psychology      1.0
food            8.0
literature     49.0
money          43.0
trivia         31.0
gossip         23.0
humor          16.0
greetings      19.0
sports         93.0
movies         29.0
science        20.0
health         10.0
dtype: float64

In [28]:
pd.get_dummies(data.tag)

,AI,computers,emotion,food,gossip,greetings,health,history,humor,literature,money,movies,politics,profile,psychology,science,sports,trivia
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
562,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
563,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
564,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
